In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc354_pure_007_bottom_218.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc354_pure_007_top_013.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc354_pure_010_bottom_116.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc354_pure_005_top_158.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc317_pure_004_bottom_069.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc317_pure_018_bottom_038.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc317_pure_016_bottom_010.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc354_pure_001_bottom_125.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc317_pure_010_bottom_173.png
/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val/pure/new_dc317_pure_014_bottom_147.png
/kaggle/input/feb-25-702010-trvaltes

In [15]:
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import roc_curve, auc, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from sklearn.metrics import roc_curve, auc, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import EfficientNetB0,MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import mixed_precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard

In [4]:
img_size = (224, 224)
batch_size = 18

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.5],
    channel_shift_range=20,
)

train_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/train"
val_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/val"
test_dir = "/kaggle/input/feb-25-702010-trvaltest/3.tr70val20te10/test"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 20827 images belonging to 2 classes.
Found 5952 images belonging to 2 classes.
Found 2975 images belonging to 2 classes.


In [19]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3),include_top = False,weights = "imagenet")
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

58889256/58889256 [==============================] - 2s 0us/step


In [20]:
for layer in base_model.layers:
    layer.trainable = False

In [21]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
callbacks1 = [
    ModelCheckpoint(filepath='model_weights.h5', save_best_only=True),
    EarlyStopping(patience=5,monitor='val_loss', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001, verbose=1)]

In [23]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=callbacks1
)

Epoch 1/20
1158/1158 [==============================] - 319s 274ms/step - loss: 0.3681 - accuracy: 0.8359 - val_loss: 0.2722 - val_accuracy: 0.8913 - lr: 0.0010
Epoch 2/20
1158/1158 [==============================] - 321s 277ms/step - loss: 0.2849 - accuracy: 0.8810 - val_loss: 0.2764 - val_accuracy: 0.8886 - lr: 0.0010
Epoch 3/20
1158/1158 [==============================] - 323s 279ms/step - loss: 0.2645 - accuracy: 0.8895 - val_loss: 0.2505 - val_accuracy: 0.9064 - lr: 0.0010
Epoch 4/20
1158/1158 [==============================] - 322s 278ms/step - loss: 0.2467 - accuracy: 0.8994 - val_loss: 0.1847 - val_accuracy: 0.9294 - lr: 0.0010
Epoch 5/20
1158/1158 [==============================] - 315s 272ms/step - loss: 0.2437 - accuracy: 0.8979 - val_loss: 0.1875 - val_accuracy: 0.9183 - lr: 0.0010
Epoch 6/20
1158/1158 [==============================] - 315s 272ms/step - loss: 0.2317 - accuracy: 0.9057 - val_loss: 0.1992 - val_accuracy: 0.9273 - lr: 0.0010
Epoch 7/20
1158/1158 [============

In [24]:
model.save('VGG16_Feb_25_702010_imagenetpretrainmodel.h5')